In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import holidays
import pycountry
import world_bank_data as wb
from region import *

In [5]:
charts = pd.read_csv(r'C:\Users\ulasz\OneDrive\Pulpit\studia\sem6\hurtownie danych\dane\charts.csv', nrows=900000)

In [8]:
charts['region'].unique()

array(['Argentina', 'Australia', 'Brazil', 'Austria', 'Belgium',
       'Colombia', 'Bolivia', 'Denmark', 'Bulgaria', 'Canada', 'Chile',
       'Costa Rica', 'Czech Republic', 'Finland', 'Dominican Republic',
       'Ecuador', 'El Salvador', 'Estonia', 'France', 'Germany', 'Global',
       'Greece', 'Guatemala', 'Honduras', 'Hong Kong', 'Hungary',
       'Iceland', 'Indonesia', 'Ireland', 'Italy', 'Japan', 'Latvia',
       'Lithuania', 'Malaysia', 'Luxembourg', 'Mexico', 'Netherlands',
       'New Zealand', 'Nicaragua', 'Norway', 'Panama', 'Paraguay', 'Peru',
       'Philippines', 'Poland', 'Portugal', 'Singapore', 'Spain',
       'Slovakia', 'Sweden', 'Taiwan', 'Switzerland', 'Turkey',
       'United Kingdom', 'United States', 'Uruguay', 'Thailand',
       'Andorra', 'Romania', 'Vietnam', 'Egypt', 'India', 'Israel',
       'Morocco', 'Saudi Arabia', 'South Africa', 'United Arab Emirates'],
      dtype=object)

In [ ]:
dim = extract_region_features(charts, 'region')

In [12]:
dim.head()

,title,rank,date,artist,url,region,chart,trend,streams,country_code,continent,language,Population
0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Argentina,top200,SAME_POSITION,253019.0,ARG,South America,Spanish,45538401.0
1,Vente Pa' Ca (feat. Maluma),2,2017-01-01,Ricky Martin,https://open.spotify.com/track/7DM4BPaS7uofFul...,Argentina,top200,MOVE_UP,223988.0,ARG,South America,Spanish,45538401.0
2,Reggaetón Lento (Bailemos),3,2017-01-01,CNCO,https://open.spotify.com/track/3AEZUABDXNtecAO...,Argentina,top200,MOVE_DOWN,210943.0,ARG,South America,Spanish,45538401.0
3,Safari,4,2017-01-01,"J Balvin, Pharrell Williams, BIA, Sky",https://open.spotify.com/track/6rQSrBHf7HlZjtc...,Argentina,top200,SAME_POSITION,173865.0,ARG,South America,Spanish,45538401.0
4,Shaky Shaky,5,2017-01-01,Daddy Yankee,https://open.spotify.com/track/58IL315gMSTD37D...,Argentina,top200,MOVE_UP,153956.0,ARG,South America,Spanish,45538401.0


In [10]:
charts[charts['region'] == 'Japan'].head(50)

,title,rank,date,artist,url,region,chart,trend,streams
4626,Closer,1,2017-01-01,"The Chainsmokers, Halsey",https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,Japan,top200,SAME_POSITION,6342.0
4627,I Feel It Coming,2,2017-01-01,"The Weeknd, Daft Punk",https://open.spotify.com/track/5GXAXm5YOmYT0kL...,Japan,top200,MOVE_UP,5697.0
4628,Let Me Love You,3,2017-01-01,"DJ Snake, Justin Bieber",https://open.spotify.com/track/4pdPtRcBmOSQDlJ...,Japan,top200,MOVE_DOWN,5149.0
4629,24K Magic,4,2017-01-01,Bruno Mars,https://open.spotify.com/track/6b8Be6ljOzmkOmF...,Japan,top200,MOVE_DOWN,4846.0
4630,Don't Wanna Know,5,2017-01-01,"Maroon 5, Kendrick Lamar",https://open.spotify.com/track/5MFzQMkrl1FOOng...,Japan,top200,MOVE_DOWN,4595.0
4631,Starboy,6,2017-01-01,"The Weeknd, Daft Punk",https://open.spotify.com/track/5aAx2yezTd8zXrk...,Japan,top200,MOVE_DOWN,4440.0
4632,Cold Water (feat. Justin Bieber & MØ),7,2017-01-01,Major Lazer,https://open.spotify.com/track/6DNtNfH8hXkqOX1...,Japan,top200,MOVE_DOWN,4154.0
4633,Wherever you are,8,2017-01-01,ONE OK ROCK,https://open.spotify.com/track/0YAMQSmHk6BSUGT...,Japan,top200,MOVE_UP,4134.0
4634,My Boo,9,2017-01-01,Shota Shimizu,https://open.spotify.com/track/2gasr6aemUqZIBB...,Japan,top200,MOVE_UP,3857.0
4635,Side To Side,10,2017-01-01,"Ariana Grande, Nicki Minaj",https://open.spotify.com/track/4pLwZjInHj3SimI...,Japan,top200,MOVE_DOWN,3838.0


In [14]:
dim.isna().sum()

title                7
rank                 0
date                 0
artist               0
url                  0
region               0
chart                0
trend                0
streams         166651
country_code         0
continent            0
language             0
Population           0
dtype: int64

In [15]:
temp = dim[dim['Population'].isna()]
temp['region'].value_counts()

Series([], Name: count, dtype: int64)

In [12]:
import pandas as pd

file_path = r'C:\Users\ulasz\OneDrive\Pulpit\studia\sem6\hurtownie danych\dane\charts.csv'
chunksize = 500_000

chunks = pd.read_csv(file_path, chunksize=chunksize)

# Zmienna do przechowywania wyników z każdego chanku
missing_regions_summary = pd.Series(dtype=int)
all_dim_chunks = []  # Jeśli chcesz potem złączyć wszystkie

for i, chunk in enumerate(chunks):

    if i < 50:
        continue
    if i > 65:
        break
    print(f"Processing chunk {i + 1}")


    dim = extract_region_features(chunk, 'region')


    all_dim_chunks.append(dim)

    missing_pop = dim[dim['Population'].isna()]
    if not missing_pop.empty:
        value_counts = missing_pop['region'].value_counts()
        missing_regions_summary = missing_regions_summary.add(value_counts, fill_value=0)

print("Brakujące wartości w kolumnie Population według regionu:")
print(missing_regions_summary.sort_values(ascending=False))




Processing chunk 51
Processing chunk 52
Processing chunk 53
Brakujące wartości w kolumnie Population według regionu:
Series([], dtype: int64)


In [13]:
full_dim_df = pd.concat(all_dim_chunks, ignore_index=True)
full_dim_df.isna().sum()

title                 0
rank                  0
date                  0
artist                0
url                   0
region                0
chart                 0
trend                 0
streams         1149291
country_code          0
continent             0
language              0
Population            0
dtype: int64

In [14]:
unknown_counts = (full_dim_df.astype(str)
                  .apply(lambda col: col.str.lower().eq('unknown'))  # eq('unknown') == (== 'unknown')
                  .sum())

# Filtrowanie tylko kolumn, w których są takie wartości
unknown_counts = unknown_counts[unknown_counts > 0]

print("Wartości 'Unknown' w kolumnach:")
print(unknown_counts)


Wartości 'Unknown' w kolumnach:
artist    47
dtype: int64


In [15]:
full_dim_df[full_dim_df['continent'] == 'Unknown']['region'].value_counts()

Series([], Name: count, dtype: int64)